In [17]:
# imports
%load_ext autoreload
%autoreload 2
from data.input_data import get_all_data
from pyomo.environ import *
from model.sets_params import define_sets_and_params
from model.variables import define_variables
from model.objective import define_objective
import model.affichage as affichage
import model.Save as save
# from model.solver_GLPK import solve_model
# from model.solver_CBC import solve_model
from model.solver_scip import solve_model
# from model.solver_scip_FAST import solve_model
# from model.solver_scip_STRONG import solve_model
from model.constraints import add_constraints
# 1. Chargement des données
data = get_all_data()
print("Données chargées avec succès.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saving matrix Distor_ihc with dimensions 20x5x4
Données chargées avec succès.


In [ ]:
# # Résolution
# # 2. Création du modèle Pyomo
# model = ConcreteModel()
# print("Modèle Pyomo créé.")
# model = define_sets_and_params(model, data)
# print("Ensembles et paramètres définis.")
# define_variables(model)
# print("Variables définies.")
# model = add_constraints(model, data)  # ou juste add_constraints(model, data)
# print("Contraintes ajoutées.")
# define_objective(model)
# print("Objectif défini.")
# # 3. Affichage des informations du modèle
# num_vars = sum(1 for v in model.component_objects(Var, active=True) for _ in v)
# num_constraints = sum(1 for c in model.component_objects(Constraint, active=True) for _ in c)

# print(f"Nombre total de variables     : {num_vars}")
# print(f"Nombre total de contraintes  : {num_constraints}")

# # Variables binaires
# num_bin_vars = sum(1 for v in model.component_objects(Var, active=True)
#                 for index in v if v[index].domain == Binary)
# print(f"Nombre de variables binaires : {num_bin_vars}")

# # Variables continues
# num_cont_vars = sum(1 for v in model.component_objects(Var, active=True)
#                     for index in v if v[index].domain == NonNegativeReals)
# print(f"Nombre de variables continues: {num_cont_vars}")

# # 3. Résolution
# result = solve_model(model, tee=True)

# # sauvegarde des resultats

# save.save_solution(model)

Modèle Pyomo créé.
Ensembles et paramètres définis.
Variables définies.
Contraintes ajoutées.
Objectif défini.
Nombre total de variables     : 28916
Nombre total de contraintes  : 58022
Nombre de variables binaires : 14112
Nombre de variables continues: 14804
✅ SCIP disponible. Lancement de la résolution avec gap = 0.15...
SCIP version 9.0.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 7.0.1] [GitHash: bebb64304e]
Copyright (c) 2002-2024 Zuse Institute Berlin (ZIB)

External libraries: 
  Soplex 7.0.1         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 1cc71921]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  MPIR 3.0.0           Multiple Precision Integers and Rationals Library developed by W. Hart (mpir.org)
  ZIMPL 3.6.0          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 690e9e7      AMPL .n

In [19]:
# Model Load
# Model Loadfrom pyomo.environ import *

# 1. Reconstruction structurelle
data = get_all_data()
model = ConcreteModel()
model = define_sets_and_params(model, data)
define_variables(model)
model = add_constraints(model, data)
define_objective(model)
save.load_solution_into_model(model)


Saving matrix Distor_ihc with dimensions 20x5x4
✅ Solution réinjectée dans le modèle Pyomo
 Valeur de la fonction objectif : 16490074.336705783


In [20]:
# Affichage Résultas
df = affichage.extract_blending_results(model)
affichage.save_blending_results_to_csv(df)
affichage.export_fusion_style_table(df, filename="S_blending_table.png")

✅ Résultats enregistrés dans blending_results_table15.csv
✅ Tableau exporté dans : S_blending_table.png


In [ ]:
df.head(0)

,N°,Commande,Qualité demandée (QM),Nom ingrédient,Stock source,Gamme,t,x_ihkt (tonnes),Real_X_ihkt,E_k,L_k,Quantity commande,Total_commande,%_blending
15,1,Commande 1,Profil_BG,MT_Smine,La mine,traitement TM (2),1,2553.2069,2297.88621,1,4,12000,11999.99996,21.276724
1,1,Commande 1,Profil_BG,TBT_Smine,La mine,traitement TM (2),1,1946.7931,1752.11379,1,4,12000,11999.99996,16.223276
16,1,Commande 1,Profil_BG,MT_Smine,La mine,traitement TM (2),2,3630.6584,3267.59256,1,4,12000,11999.99996,30.255487
31,1,Commande 1,Profil_BG,BT_Se,Stock épierré,sans traitement (0),2,2000.0000,2000.00000,1,4,12000,11999.99996,16.666667
24,1,Commande 1,Profil_BG,BT_Sc1,Stock criblé 1,sans traitement (0),3,2682.4074,2682.40740,1,4,12000,11999.99996,22.353395
26,2,Commande 2,Export,MT_Sc1,Stock criblé 1,TM + Laverie (3),4,3392.7879,2714.23032,3,6,5000,5000.00001,67.855758
4,2,Commande 2,Export,PBG_Smine,La mine,traitement TM (2),5,2539.7441,2285.76969,3,6,5000,5000.00001,50.794882
17,3,Commande 3,Tess,MT_Smine,La mine,traitement TM (2),5,500.0000,450.00000,5,9,5000,5000.00000,10.000000
32,3,Commande 3,Tess,PBG_Se,Stock épierré,sans traitement (0),7,4550.0000,4550.00000,5,9,5000,5000.00000,91.000000
5,4,Commande 4,BG_BT,PBG_Smine,La mine,traitement TM (2),7,1055.5556,950.00004,7,12,5000,5000.00004,21.111112


In [22]:
# Affichage Teneurs
df_t = affichage.extract_tenurs_results(model)
df_t.head()

,Commande,Qualité demandée (QM),mu_ck,D_k,Cible_BPL,Teneur_BPL,Cible_MgO,Teneur_MgO,Cible_SiO2,Teneur_SiO2,Cible_Cd,Teneur_Cd
0,Commande 1,Profil_BG,1.011540e+06,12000,58.995,58.995,1.3,1.300000,12.75,12.750000,11.25,11.250000
1,Commande 2,Export,4.390248e+05,5000,65.500,65.500,0.8,0.795657,6.40,6.464715,15.00,15.044591
2,Commande 3,Tess,4.119125e+05,5000,58.995,59.130,1.3,1.300000,12.75,12.750000,9.00,9.202500
3,Commande 4,BG_BT,4.090000e+05,5000,58.500,58.500,0.8,0.800000,6.50,6.500000,16.00,16.000000
4,Commande 5,Stand,4.177250e+05,5000,58.995,58.995,1.3,1.300000,12.75,12.750000,10.50,10.500000


In [23]:
# df.head(50)  # Affichage des premières lignes du DataFrame

In [24]:
# affichage.export_fusion_style_table(df, filename="S_blending_table.png")

In [25]:
# df.head(10)  # Affiche les premières lignes du DataFrame pour vérification

In [26]:
#  FLUX MINE
def diagnostic_flux_mine(model):
    print("===== DIAGNOSTIC FLUX MINE =====")
    for i in model.QS_mines:
        total = 0
        for h in model.H:
            for k in model.K:
                for t in model.T:
                    if (i, h, k, t) in model.IHKT_valid:
                        val = model.x_ihkt[i, h, k, t].value
                        if val is not None:
                            total += val
        print(f"Ingrédient {i} (Mine): Total extrait sur tout l’horizon = {total}")
        print(f"Stock initial = {model.Stock_initial_i[i]}")
diagnostic_flux_mine(model)

===== DIAGNOSTIC FLUX MINE =====
Ingrédient 1 (Mine): Total extrait sur tout l’horizon = 4799.0464999999995
Stock initial = 1113900
Ingrédient 2 (Mine): Total extrait sur tout l’horizon = 0.0
Stock initial = 5000
Ingrédient 3 (Mine): Total extrait sur tout l’horizon = 33612.7348
Stock initial = 21400
Ingrédient 4 (Mine): Total extrait sur tout l’horizon = 14178.3197
Stock initial = 4600


In [27]:
# 1. Demande totale
demande_totale = sum([model.D_k[k] for k in model.K])
print(f"Demande totale sur tout l'horizon (somme D_k) : {demande_totale:.0f} tonnes")

# 2. Stock initial total
stock_initial_total = sum([model.Stock_initial_i[i] for i in model.I])
print(f"Stock initial total (somme Stock_initial_i) : {stock_initial_total:.0f} tonnes")

# 3. Affiche le ratio
if demande_totale > stock_initial_total:
    print(f"\n La demande ({demande_totale:.0f}) est supérieure au stock initial ({stock_initial_total:.0f}) !")
else:
    print(f"\nLa demande ({demande_totale:.0f}) est couverte par le stock initial ({stock_initial_total:.0f})")


Demande totale sur tout l'horizon (somme D_k) : 77000 tonnes
Stock initial total (somme Stock_initial_i) : 1242800 tonnes

La demande (77000) est couverte par le stock initial (1242800)


In [28]:
# Somme des stocks initiaux pour QS_Sc1
stock_QS_Sc1 = sum([model.Stock_initial_i[i] for i in model.QS_Sc1])
print(f"Stock initial total dans QS_Sc1 : {stock_QS_Sc1:.0f} tonnes")

# Somme des stocks initiaux pour QS_Se
stock_QS_Se = sum([model.Stock_initial_i[i] for i in model.QS_Se])
print(f"Stock initial total dans QS_Se  : {stock_QS_Se:.0f} tonnes")

# Somme des stocks initiaux pour QSL_Sf1
stock_QSL_Sf1 = sum([model.Stock_initial_i[i] for i in model.QSL_Sf1])
print(f"Stock initial total dans QSL_Sf1 : {stock_QSL_Sf1:.0f} tonnes")

# Optionnel : total cumulé sur les trois
total_stocks = stock_QS_Sc1 + stock_QS_Se + stock_QSL_Sf1
print(f"Somme totale des stocks initiaux utilisés (QS_Sc1 + QS_Se + QSL_Sf1) : {total_stocks:.0f} tonnes")

# 1. Demande totale
demande_totale = sum([model.D_k[k] for k in model.K])
print(f"Demande totale sur tout l'horizon (somme D_k) : {demande_totale:.0f} tonnes")
# 3. Affiche le ratio
if demande_totale > stock_initial_total:
    print(f"\n La demande ({demande_totale:.0f}) est supérieure au stock initial ({total_stocks:.0f}) !")
else:
    print(f"\nLa demande ({demande_totale:.0f}) est couverte par le stock initial ({total_stocks:.0f})")



Stock initial total dans QS_Sc1 : 31000 tonnes
Stock initial total dans QS_Se  : 20300 tonnes
Stock initial total dans QSL_Sf1 : 16000 tonnes
Somme totale des stocks initiaux utilisés (QS_Sc1 + QS_Se + QSL_Sf1) : 67300 tonnes
Demande totale sur tout l'horizon (somme D_k) : 77000 tonnes

La demande (77000) est couverte par le stock initial (67300)


In [29]:
# print("===== CONTRÔLE DES FLUX D’UTILISATION =====")
# for i in model.I:
#     initial = model.Stock_initial_i[i]
#     extrait = sum(model.x_ihkt[i, h, k, t].value for h in model.H for k in model.K for t in model.T if (i, h, k, t) in model.IHKT_valid)
#     final = model.S_it[i, model.T.last()].value
#     print(f"Ingrédient {i:2} | Stock initial: {initial:8.0f} | Extrait total: {extrait:8.0f} | Stock final: {final:8.0f} | Flux total = {initial + sum(model.Ait[t, i] for t in model.T) - final:8.0f}")


In [30]:
import pandas as pd
import model.affichage as affichage


In [31]:
df_flux = affichage.Stocks_utilisation(model)
df_flux.head(5) 
# df_flux.tail()

,Ingrédient,Stock initial,Extrait total,Ajout total,Flux Sortant,Flux Entrant,Stock final
0,TBT_Smine (1),1113900,4799.05,112000,1109100.95,112000,1221100.95
1,BT_Smine (2),5000,0.00,84000,5000.00,84000,89000.00
2,PBG_Smine (3),21400,33612.73,84000,-12212.73,84000,71787.27
3,MT_Smine (4),4600,14178.32,56000,-9578.32,56000,46421.68
4,TBT_Sc1 (5),3000,3000.00,0,0.00,0,0.00


In [32]:
df_util = affichage.compute_resource_utilization(model, data)
# df_util.to_csv("resource_utilization.csv", index=False)
print(df_util.shape)  # aperçu
df_util.head(10) # aperçu

(140, 5)


,Période,Ressource,Flux_total,Capacité,Taux_utilisation
0,1,1,4500.00000,4500.0,1.000000
1,1,2,0.00000,3600.0,0.000000
2,1,3,0.00000,3600.0,0.000000
3,1,4,4050.00000,4750.0,0.852632
4,1,5,4050.00000,7600.0,0.532895
5,2,1,3630.65840,4500.0,0.806813
6,2,2,0.00000,3600.0,0.000000
7,2,3,0.00000,3600.0,0.000000
8,2,4,3267.59256,4750.0,0.687914
9,2,5,5267.59256,7600.0,0.693104


In [36]:
df.head(60)

,N°,Commande,Qualité demandée (QM),Nom ingrédient,Stock source,Gamme,t,x_ihkt (tonnes),Real_X_ihkt,E_k,L_k,Quantity commande,Total_commande,%_blending
15,1,Commande 1,Profil_BG,MT_Smine,La mine,traitement TM (2),1,2553.2069,2297.88621,1,4,12000,11999.99996,21.276724
1,1,Commande 1,Profil_BG,TBT_Smine,La mine,traitement TM (2),1,1946.7931,1752.11379,1,4,12000,11999.99996,16.223276
16,1,Commande 1,Profil_BG,MT_Smine,La mine,traitement TM (2),2,3630.6584,3267.59256,1,4,12000,11999.99996,30.255487
31,1,Commande 1,Profil_BG,BT_Se,Stock épierré,sans traitement (0),2,2000.0000,2000.00000,1,4,12000,11999.99996,16.666667
24,1,Commande 1,Profil_BG,BT_Sc1,Stock criblé 1,sans traitement (0),3,2682.4074,2682.40740,1,4,12000,11999.99996,22.353395
26,2,Commande 2,Export,MT_Sc1,Stock criblé 1,TM + Laverie (3),4,3392.7879,2714.23032,3,6,5000,5000.00001,67.855758
4,2,Commande 2,Export,PBG_Smine,La mine,traitement TM (2),5,2539.7441,2285.76969,3,6,5000,5000.00001,50.794882
17,3,Commande 3,Tess,MT_Smine,La mine,traitement TM (2),5,500.0000,450.00000,5,9,5000,5000.00000,10.000000
32,3,Commande 3,Tess,PBG_Se,Stock épierré,sans traitement (0),7,4550.0000,4550.00000,5,9,5000,5000.00000,91.000000
5,4,Commande 4,BG_BT,PBG_Smine,La mine,traitement TM (2),7,1055.5556,950.00004,7,12,5000,5000.00004,21.111112


In [34]:
# %load_ext autoreload
# %autoreload 2
# from data.input_data import get_all_data
# from pyomo.environ import *
# from model.sets_params import define_sets_and_params
# from model.variables import define_variables
# from model.objective import define_objective
# # from model.solver_GLPK import solve_model
# # from model.solver_CBC import solve_model
# from model.solver_scip import solve_model
# from model.constraints_copy import add_constraints
# # 1. Chargement des données
# data = get_all_data()
# # 2. Création du modèle Pyomo
# model = ConcreteModel()
# model = define_sets_and_params(model, data)
# define_variables(model)
# model = add_constraints(model, data)  # ou juste add_constraints(model, data)
# define_objective(model)

In [35]:
# model.test=Param(model.T, initialize=0)  # Exemple d'initialisation